# Agent
Agent的核心思想是使用LLM作为推理的大脑，以制定解决问题的计划、借助工具实施动作。

首先，尝试用LLM进行数学计算：

In [2]:
import shutup
shutup.please()

from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model='gpt-3.5-turbo' ,temperature=0)  # 这里不使用gpt的话，内置agent部分会报错

llm.predict('请计算 3.14^6.5')  # 1698.41

'答案为1191.75070741'

可以看到直接计算的结果非常离谱，因为大模型本质是上文本补全的过程，计算能力是比较差的。

现在我们使用agent的思路解决这个问题：

In [3]:
from langchain.agents import load_tools, initialize_agent, get_all_tool_names

tools = load_tools(['llm-math'], llm=llm)
agent = initialize_agent(tools, llm=llm)

agent.run('3.14^6.5')

'1698.4106814274528'

可以看到正确输出了结果，我们再来看看有哪些内置tool：

In [4]:
get_all_tool_names()

['requests',
 'requests_get',
 'requests_post',
 'requests_patch',
 'requests_put',
 'requests_delete',
 'terminal',
 'sleep',
 'wolfram-alpha',
 'google-search',
 'google-search-results-json',
 'searx-search-results-json',
 'bing-search',
 'metaphor-search',
 'ddg-search',
 'google-lens',
 'google-serper',
 'google-scholar',
 'google-finance',
 'google-trends',
 'google-jobs',
 'google-serper-results-json',
 'searchapi',
 'searchapi-results-json',
 'serpapi',
 'dalle-image-generator',
 'twilio',
 'searx-search',
 'merriam-webster',
 'wikipedia',
 'arxiv',
 'golden-query',
 'pubmed',
 'human',
 'awslambda',
 'stackexchange',
 'sceneXplain',
 'graphql',
 'openweathermap-api',
 'dataforseo-api-search',
 'dataforseo-api-search-json',
 'eleven_labs_text2speech',
 'google_cloud_texttospeech',
 'reddit_search',
 'news-api',
 'tmdb-api',
 'podcast-api',
 'memorize',
 'llm-math',
 'open-meteo-api']

自定义工具解决实际问题：

In [5]:
from langchain.agents import initialize_agent
from datetime import datetime
from langchain.tools import Tool

def get_current_time(query):
    return datetime.now()

tools = [Tool(
    name='get_current_time',
    func=get_current_time,
    description='获取当前日期时间'
)]

agent = initialize_agent(tools, llm=llm, verbose=True)
result = agent.run('今天是几号？')
result



> Entering new AgentExecutor chain...
I need to find out the current date.
Action: get_current_time
Action Input: 
Observation: 2024-11-29 18:00:34.012581
Thought:The date is included in the current date and time information.
Final Answer: 今天是29号。

> Finished chain.


'今天是29号。'

Action Input是空，因为我们直接问了今天是几号，而模型不知道今天是几号，Observation是调用函数后的输出结果。

最后Thought思考的时候过滤出了正确答案。

通俗解释，Agent 就是将模型进行封装，使得它可以通过用户的输入，理解用户的意图，然后执行一个特定的动作。